In [6]:
!pip3 install --user tensorflow=='2.0.0a0'
!pip3 install sklearn

  Using cached https://files.pythonhosted.org/packages/9f/4e/402a4d7f21d03cd640b41376727a9ea2bac5a06d2556c04517878dad0c3d/tensorflow-2.0.0a0-cp37-cp37m-manylinux1_x86_64.whl


  Found existing installation: tensorflow 1.14.0


    Uninstalling tensorflow-1.14.0:


      Successfully uninstalled tensorflow-1.14.0


  The scripts freeze_graph, saved_model_cli, tensorboard, tf_upgrade_v2, tflite_convert, toco and toco_from_protos are installed in '/projects/d19a8e8b-8443-4e4b-86b0-93ceef78058e/.sage//local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [18]:
import os
import joblib

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Activation, Bidirectional, Dropout, TimeDistributed
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import accuracy_score, classification_report

np.random.seed(seed=17)

In [2]:
# Import simulation results
results = pd.read_csv('processed_results_summary.csv', keep_default_na=False)
results.head()

,Unnamed: 0,Simulation no#,Line,Load,Wind1,Wind2,Wind3,flag,Sequence,n_failures,t_fail,t_first_fail,dt_fail,failed component_type,failed_component_type,cascade,fail
0,0,1,Line 04 - 05,0.7,0.0,0.0,0.0,0,[],0,,,,,,False,False
1,18,10,Line 26 - 28,0.7,0.0,0.0,0.0,0,[],0,,,,,,False,False
2,198,100,Line 09 - 39,0.7,0.0,0.0,0.4,0,[],0,,,,,,False,False
3,1998,1000,Line 22 - 23,0.7,0.0,0.8,1.0,0,[],0,,,,,,False,False
4,19998,10000,Line 10 - 11,0.8,0.4,0.2,0.0,0,[],0,,,,,,False,False


### Data Generator

- Create a data generator to read in training, validation and test data by batches
- Model input formats:  
    x -> (batch_size, n_time_steps, n_features)  
    y -> (batch_size, 1)

In [3]:
DATA_PATH = '/output/02_preprocessed_numpy'
files = os.listdir(DATA_PATH)

In [4]:
# Get the simulation ids of sims with and without failures
fail_ids = results[results['flag']==1]['Simulation no#'].tolist()
non_fail_ids = results[results['flag']==0]['Simulation no#'].tolist()

# Randomly select fail and non-fail ids
fail_ids = np.random.choice(fail_ids, size=len(fail_ids), replace=False)
non_fail_ids = np.random.choice(non_fail_ids, size=20000, replace=False)

# Get full filepaths
paths_fail, paths_non_fail = [], []
labels_fail, labels_non_fail = [], []

for file in files:
    file_id = int(file.split('.pkl')[0])

    if file_id in fail_ids and len(paths_fail) < 3500:
        paths_fail.append(os.path.join(DATA_PATH, file))
        labels_fail.append(1)

    if file_id in non_fail_ids and len(paths_non_fail) < 3500:
        paths_non_fail.append(os.path.join(DATA_PATH, file))
        labels_non_fail.append(0)

print(len(paths_fail), len(paths_non_fail))
filepaths, labels = [None]*7000, [None]*7000
filepaths[::2], labels[::2] = paths_fail, labels_fail
filepaths[1::2], labels[1::2] = paths_non_fail, labels_non_fail

train_paths, train_labels = filepaths[:6000], labels[:6000]
val_paths, val_labels = filepaths[6000:6500], labels[6000:6500]
test_paths, test_labels = filepaths[6500:], labels[6500:]

3500 3500


In [14]:
def data_generator(batch_size, filepaths, labels, length, shift):
    count = 0
    X, y = [], []

    for file, label in zip(filepaths, labels):
        # Load simulation file as numpy array
        arr = joblib.load(os.path.join(DATA_PATH, file))

        # Windowing - create evenly sized samples
        end_index = arr.shape[0] - length

        for i in range(0, end_index, shift):
            y.append(int(label))
            X.append(arr[i:(i+length), 1:])
            count += 1

            if count >= batch_size:
                yield (np.array(X).astype(np.float32), np.array(y))
                X, y = [], []
                count = 0

### Model Training and Evaluation
- Train a single layer LSTM model
- Evaluate on test data: accuracy, precision, recall, F1 score

In [20]:
# LSTM
EPOCH_SIZE = 10

# Number of time steps and number of features
n_time_steps = 100
n_feats = 259

model = Sequential()
model.add(LSTM(50, input_shape = (n_time_steps, n_feats)))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Precision(), Recall()])
model.summary()

# Start training
early_stop = EarlyStopping(monitor='val_loss', patience=3)
checkpoint = ModelCheckpoint(
    filepath='checkpoint',
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True
)

model.fit_generator(
    data_generator(256, train_paths, train_labels, 100, 25),
    validation_data=data_generator(256, val_paths, val_labels, 100, 25),
    validation_steps=6,
    epochs=EPOCH_SIZE,
    steps_per_epoch=100,
    callbacks=[early_stop, checkpoint]
)

model.save('model')

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
unified_lstm_8 (UnifiedLSTM) (None, 50)                62000     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 51        
Total params: 62,051
Trainable params: 62,051
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


  1/100 [..............................] - ETA: 3:28 - loss: 0.7416 - accuracy: 0.4375 - precision: 0.0000e+00 - recall: 0.0000e+00

  2/100 [..............................] - ETA: 1:48 - loss: 0.7021 - accuracy: 0.4434 - precision: 0.3832 - recall: 0.1577        

  3/100 [..............................] - ETA: 1:14 - loss: 0.6986 - accuracy: 0.5716 - precision: 0.6558 - recall: 0.3561

  4/100 [>.............................] - ETA: 1:10 - loss: 0.7117 - accuracy: 0.5195 - precision: 0.5357 - recall: 0.4038

...

 96/100 [===========================>..] - ETA: 2s - loss: 0.0656 - accuracy: 0.9782 - precision: 0.9796 - recall: 0.9768

 97/100 [============================>.] - ETA: 2s - loss: 0.0666 - accuracy: 0.9772 - precision: 0.9775 - recall: 0.9770

 98/100 [============================>.] - ETA: 1s - loss: 0.0670 - accuracy: 0.9771 - precision: 0.9777 - recall: 0.9765

 99/100 [============================>.] - ETA: 0s - loss: 0.0665 - accuracy: 0.9774 - precision: 0.9780 - recall: 0.9767

100/100 [==============================] - 80s 797ms/step - loss: 0.0663 - accuracy: 0.9775 - precision: 0.9781 - recall: 0.9769 - val_loss: 0.6092 - val_accuracy: 0.6758 - val_precision: 0.6029 - val_recall: 1.0000


In [25]:
test_y, pred_y = [], []

for X, y in data_generator(256, test_paths, test_labels, 100, 25):
    test_y.append(y)
    pred_y.append(model.predict(X))

test_y = np.concatenate(test_y)
pred_y = np.concatenate(pred_y)
pred_y[pred_y >= 0.5] = 1
pred_y[pred_y < 0.5] = 0

# Accuracy, precision, recall, F1 score
acc = accuracy_score(test_y, pred_y)

print('Accuracy on test set: {:.3f}'.format(acc))
print(classification_report(test_y, pred_y))

Accuracy on test set: 0.924
              precision    recall  f1-score   support

           0       0.97      0.87      0.92      8956
           1       0.88      0.98      0.93      8964

    accuracy                           0.92     17920
   macro avg       0.93      0.92      0.92     17920
weighted avg       0.93      0.92      0.92     17920

